# Imports

In [1]:
import numpy as np
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Preprocessing

In [2]:
#========== IMPORTS =============#
# Allows jupyter notebook to be imported
import jupyter_import
from data_preproc.Preprocess import preprocess
#================================#

importing Jupyter notebook from D:\FYP_git\FYP\Program\Algorithm\data_preproc\Preprocess.ipynb


# Feature Selection

In [3]:
from data_preproc.CFS import cfs_algo
from data_preproc.RFE import rfe_algo
from data_preproc.RR import ridge_algo

importing Jupyter notebook from D:\FYP_git\FYP\Program\Algorithm\data_preproc\CFS.ipynb
importing Jupyter notebook from D:\FYP_git\FYP\Program\Algorithm\data_preproc\RFE.ipynb
importing Jupyter notebook from D:\FYP_git\FYP\Program\Algorithm\data_preproc\RR.ipynb


# Algorithms

## Base Predictors

In [4]:
from pred_mdls.base.Complement_Naive_Bayes import complement_naive_bayes_model
from pred_mdls.base.Decision_Tree import decision_tree_model
from pred_mdls.base.Logistic_Regression import logistic_regression_model
from pred_mdls.base.Multi_Layer_Perceptron import multi_layer_perceptron_model
from pred_mdls.base.Naive_Bayes import naive_bayes_model

importing Jupyter notebook from D:\FYP_git\FYP\Program\Algorithm\pred_mdls\base\Complement_Naive_Bayes.ipynb
importing Jupyter notebook from D:\FYP_git\FYP\Program\Algorithm\pred_mdls\base\Decision_Tree.ipynb
importing Jupyter notebook from D:\FYP_git\FYP\Program\Algorithm\pred_mdls\base\Logistic_Regression.ipynb
importing Jupyter notebook from D:\FYP_git\FYP\Program\Algorithm\pred_mdls\base\Multi_Layer_Perceptron.ipynb
importing Jupyter notebook from D:\FYP_git\FYP\Program\Algorithm\pred_mdls\base\Naive_Bayes.ipynb


## Ensemble Predictors

In [5]:
from pred_mdls.ensemble.Random_Forest import random_forest_model
from pred_mdls.ensemble.Rotation_Forest import rotation_forest_model
from pred_mdls.ensemble.Voting import voting_model

importing Jupyter notebook from D:\FYP_git\FYP\Program\Algorithm\pred_mdls\ensemble\Random_Forest.ipynb
importing Jupyter notebook from D:\FYP_git\FYP\Program\Algorithm\pred_mdls\ensemble\Rotation_Forest.ipynb
importing Jupyter notebook from D:\FYP_git\FYP\Program\Algorithm\pred_mdls\ensemble\Voting.ipynb


In [6]:
def data_conversion(data):
    for i in range(len(data)):
        if data[i] == b'N':
            data[i] = 0
        else:
            data[i] = 1
    return data

def read_data(filename):
    data = arff.loadarff(filename)
    loaddata = pd.DataFrame(data[0])
    return loaddata

def process_data(loaddata,features):
    # Features are selected based on CFS
    software_metrics = np.array(loaddata[features])
    labels = np.array(loaddata['Defective'])
    return software_metrics,labels

def train_data(software_metrics,labels):
    X_train, X_test, y_train, y_test = train_test_split(software_metrics, labels, test_size = 0.1)
    y_train = y_train.astype('str')
    y_test = y_test.astype('str')
    return X_train, X_test, y_train, y_test

#def feature_selection(fs,X):
#    dfscores = pd.DataFrame(fs.scores_)
#    dfcolumns = pd.DataFrame(X.columns)
#    feature_scores = pd.concat([dfcolumns,dfscores.fillna(0)],axis=1)
#    feature_scores.columns = ['Feature','Score']
#    best_ten_features = feature_scores.nlargest(10,'Score')
#    return best_ten_features

# def feature_selection_algo(X,Y):
#     array = []
#     data = [X,Y]
#     cfs = cfs_algo(data)
#     ridge = ridge_algo(data)
#     print(f'Ridge score: {ridge.get_support()}')
    # rfe = rfe_algo(data)
    # print(f'RFE score: {rfe.score}')

def main(filename):
    # Read the file
    loaddata = read_data(filename)
    software_metrics = loaddata.iloc[:,:-1] #Software metrics
    labels = loaddata.iloc[:,-1] #Labels

    X_train, X_test, y_train, y_test = train_data(software_metrics,labels)

    # ===== Feature Selection ====== #

    # ==== CFS ==== #
    data = [X_train,y_train]
    cfs = cfs_algo(data)
    cfs_best_ten_features = feature_selection(cfs,software_metrics)
    print(cfs_best_ten_features)
    # ============= #

    # ===== Ridge ===== #
    software_metrics = np.array(loaddata.iloc[:,:-1]) #Software metrics
    labels = data_conversion(np.array(loaddata.iloc[:,-1])) #label 

    X_train, X_test, y_train, y_test = train_data(software_metrics,labels)

    data = [X_train,y_train]
    ridge = ridge_algo(data)
    # print(ridge.intercept_)
    # print(ridge.score(software_metrics,labels))
    # features = best_ten_features['Feature'].values
    # ===================== #

    # ===== RFE ======== #
    rfe = rfe_algo(data)
    print(rfe.support_)

    # software_metrics,labels = process_data(loaddata,features)

    # X_train, X_test, y_train, y_test = train_data(software_metrics,labels)

    # Preprocessing
    preprocessed_data = preprocess(filename)

    # Algorithms
    # return preprocessed_data

if __name__=='__main__':
    filename = 'datasets/KC4.arff.txt'
    main(filename)

                            Feature      Score
27             MAINTENANCE_SEVERITY  29.343682
2                        CALL_PAIRS  27.311244
31  NORMALIZED_CYLOMATIC_COMPLEXITY   9.591940
30                       NODE_COUNT   8.523168
12                       EDGE_COUNT   8.429404
1                      BRANCH_COUNT   7.672070
6             CYCLOMATIC_COMPLEXITY   7.672070
11                   DESIGN_DENSITY   7.432104
10                DESIGN_COMPLEXITY   7.367117
14                ESSENTIAL_DENSITY   1.074798


D:\Anaconda\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:115: UserWarning: Features [ 0  3  4  5  8  9 15 16 17 18 19 20 21 22 23 24 25 26 28 29 32 33 34 35
 37 38] are constant.
  UserWarning)
D:\Anaconda\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


[False  True  True False False False  True  True False False  True  True
 False  True  True False False False False False False False False False
 False False False  True False False False  True False False False False
 False False False False]
